# Problem 51: Prime digit replacements
By replacing the 1st digit of the 2-digit number *3, it turns out that six of the nine possible values: 13, 23, 43, 53, 73, and 83, are all prime.

By replacing the 3rd and 4th digits of 56**3 with the same digit, this 5-digit number is the first example having seven primes among the ten generated numbers, yielding the family: 56003, 56113, 56333, 56443, 56663, 56773, and 56993. Consequently 56003, being the first member of this family, is the smallest prime with this property.

Find the smallest prime which, by replacing part of the number (not necessarily adjacent digits) with the same digit, is part of an eight prime value family.

In [1]:
def prime_sieve(n):
    '''
    Array of prime truth values for 
    integers from 0 to n (inclusive)
    '''
    if n <= 2: return [False]*(n+1)
    sieve = [True] * (n+1)
    sieve[:2] = [False] * 2
    for i in range(2, int(n**0.5)+1):
        sieve[i**2 : n+1 : i] = [False] * len(range(i**2,n+1,i))
    return sieve

In [2]:
def idx_choices(length, outer=True):
    '''
    Generator of all non-empty subsets of
    integers between 0 and length (inclusive)
    '''
    if length == 0:
        yield []
    else:
        for choice in idx_choices(length-1, False):
            yield [length-1] + choice
            if not outer or len(choice) > 0:
                yield choice

In [3]:
def prime_digit_replacements(n, limit):
    '''
    Smallest number in prime number family of size n,
    related to each other by the replacement of identical
    digits and all less than limit
    '''
    base = 10
    primes = prime_sieve(limit)
    for N in range(11, limit, 2):
        if not primes[N]:
            continue
        string = str(N)
        threshold = base - n
        
        # iterate over all digit subsets
        for choice in idx_choices(len(string)-1):
            digit = string[choice[0]]
            if int(digit) > threshold:
                continue
            # check digits in subset are identical
            for i in range(1, len(choice)):
                if string[choice[i]] != digit:
                    break
            else:
                non_primes = int(digit)
                for d in range(int(digit)+1, base):
                    new_string = list(string)
                    for idx in choice:
                        new_string[idx] = str(d)
                    new_string = ''.join(new_string)
                    if not primes[int(new_string)]:
                        non_primes += 1
                    if non_primes > threshold:
                        break
                    if (d - non_primes + 1) == n:
                        return N
    
    print('No solutions found within limit')
    return

In [4]:
prime_digit_replacements(8, 10**6)
# should return 121313

121313

# Problem 52: Permuted multiples
It can be seen that the number, 125874, and its double, 251748, contain exactly the same digits, but in a different order.

Find the smallest positive integer, x, such that 2x, 3x, 4x, 5x, and 6x, contain the same digits.

In [5]:
def permuted_multiples(n, m):
    '''
    Smallest positive integer x (with at most m digits)
    s.t. 2*x, 3*x,..., n*x are all permutations of x
    '''
    b = 10
    max_sig = b//n + 1
    for n_digits in range(m):
        for N in range(b**n_digits, max_sig * b**n_digits):
            sorted_N = sorted(str(N))
            for i in range(2, n+1):
                if sorted_N != sorted(str(i * N)):
                    break
            else:
                return N
    print('No solutions found within range')
    return

In [6]:
permuted_multiples(6, 6)
# should return 142857

142857

# Problem 53: Combinatoric selections
There are exactly ten ways of selecting three from five, 12345:

123, 124, 125, 134, 135, 145, 234, 235, 245, and 345

In combinatorics, we use the notation, $^5C_3 = 10$.

In general,

$^nC_r = \cfrac{n!}{r!(n−r)!}$, $r \leq n$, 

$n! = n×(n−1)×...×3×2×1$, and $0! = 1$.

It is not until $n = 23$, that a value exceeds one-million: $^{23}C_{10} = 1144066$. How many, not necessarily distinct, values of  $^nC_r$, for $1 \leq n \leq 100$, are greater than one-million?

In [7]:
from scipy.special import comb

In [8]:
def comb_selections(a, b):
    '''
    Number of values of nCr which exceed a
    for n between 1 and b (inclusive)
    '''
    tot = 0
    
    # check all unique nCr values in descending order
    for n in reversed(range(1, b+1)):
        start = n//2 + n%2
        if comb(n, start, True) <= a:
            break
        tot += (1 + n%2)
        for r in range(start+1, n+1):
            if comb(n, r, True) <= a:
                break
            tot += 2
            
    return tot

In [9]:
comb_selections(10**6, 100)
# should return 4075

4075

# Problem 54: Poker hands
In the card game poker, a hand consists of five cards and are ranked, from lowest to highest, in the following way:

- High Card: Highest value card.

- One Pair: Two cards of the same value.

- Two Pairs: Two different pairs.

- Three of a Kind: Three cards of the same value.

- Straight: All cards are consecutive values.

- Flush: All cards of the same suit.

- Full House: Three of a kind and a pair.

- Four of a Kind: Four cards of the same value.

- Straight Flush: All cards are consecutive values of same suit.

- Royal Flush: Ten, Jack, Queen, King, Ace, in same suit.

The cards are valued in the order: 

2, 3, 4, 5, 6, 7, 8, 9, 10, Jack, Queen, King, Ace.

If two players have the same ranked hands then the rank made up of the highest value wins; for example, a pair of eights beats a pair of fives (see example 1 below). But if two ranks tie, for example, both players have a pair of queens, then highest cards in each hand are compared (see example 4 below); if the highest cards tie then the next highest cards are compared, and so on.

Consider the following five hands dealt to two players:

|Hand| Player 1 | Player 2 | Winner |
|-|-|-|-|
|1 | 5H 5C 6S 7S KD (Pair of Fives) | 2C 3S 8S 8D TD (Pair of Eights) | Player 2 |
|2 |5D 8C 9S JS AC (Highest card Ace) | 2C 5C 7D 8S QH (Highest card Queen)| Player 1|
|3 | 2D 9C AS AH AC (Three Aces)| 3D 6D 7D TD QD (Flush with Diamonds)| Player 2|
|4 |4D 6S 9H QH QC (Pair Queens High 9)|3D 6D 7H QD QS (Pair Queens High 7)| Player 1|
|5 | 2H 2D 4C 4D 4S (Full House: Three 4s)|3C 3D 3S 9S 9D (Full House: Three 3s)| Player 1|                 

The file, data > poker.txt, contains one-thousand random hands dealt to two players. Each line of the file contains ten cards (separated by a single space): the first five are Player 1's cards and the last five are Player 2's cards. You can assume that all hands are valid (no invalid characters or repeated cards), each player's hand is in no specific order, and in each hand there is a clear winner. How many hands does Player 1 win?

In [10]:
import numpy as np

In [11]:
# load poker hands
from pathlib import Path
poker_path = Path('data') / 'poker.txt'
poker_file = open(poker_path, 'r')
poker = poker_file.readlines()
for i in range(len(poker)):
    poker[i] = poker[i].replace('\n','').split()

In [12]:
val_order = ['2','3','4','5','6','7','8','9','T','J','Q','K','A']
rank_order = ['HC', '1P', '2P', '3K', 'S', 'F', 'FH', '4K', 'SF']

In [13]:
def sort_hand(hand):
    '''
    Sort hand in order of value
    '''
    key = lambda card : val_order.index(card[0])
    return sorted(hand, key=key)

In [14]:
def parse_hand(hand):
    '''
    Value indices and suits of cards in hand
    '''
    hand = sort_hand(hand)
    vals = []
    suits = []
    for card in hand:
        vals += [val_order.index(card[0])]
        suits += card[1]
    return vals, suits

In [15]:
def evaluate_hand(hand):
    '''
    Rank index, rank value index and
    highest value index in hand
    '''
    vals, suits = parse_hand(hand)
    rank = 'HC'
    straight = flush = True
    repeated_vals = []
    
    # check for duplicate values
    for i in range(1, len(hand)):
        if suits[i] != suits[i-1]:
            flush = False
        if vals[i] != (vals[i-1] + 1):
            straight = False
        if vals[i] == vals[i-1]:
            if vals[i] in repeated_vals:
                if rank == '3K':
                    rank = '4K'
                elif rank == '2P':
                    rank = 'FH'
                elif rank == '1P':
                    rank = '3K'
            else:
                repeated_vals += [vals[i]]
                if rank == '3K':
                    rank = 'FH'
                elif rank == '1P':
                    rank = '2P'
                else:
                    rank = '1P'
    
    # check for straight/flush
    if straight and flush:
        rank = 'SF'
    elif straight:
        rank = 'S'
    elif flush:
        rank = 'F'
    
    rank_idx = rank_order.index(rank)
    if repeated_vals and (not flush):
        rank_val_idx = repeated_vals[-1]
    else:
        rank_val_idx = vals[-1]
    
    
    return rank_idx, rank_val_idx, vals[-1]

In [16]:
def compare_hands(h1, h2):
    '''
    1 if hand h1 beats hand h2,
    -1 if h2 beats h1, 0 if tie
    '''
    r1, r2 = evaluate_hand(h1), evaluate_hand(h2)
    if r1 > r2:
        return 1
    elif r1 < r2:
        return -1
    else:
        return 0

In [17]:
def player1_wins(games):
    '''
    Number of hands won by player 1 across games
    '''
    wins = 0
    for g in games:
        # separate hands
        h1, h2 = g[:5], g[5:]
        # compare
        if compare_hands(h1, h2) == 1:
            wins += 1
    return wins

In [18]:
player1_wins(poker)
# should return 376

376

# Problem 55: Lychrel numbers
If we take 47, reverse and add, 47 + 74 = 121, which is palindromic.

Not all numbers produce palindromes so quickly. For example,

349 + 943 = 1292,

1292 + 2921 = 4213

4213 + 3124 = 7337

That is, 349 took three iterations to arrive at a palindrome.

Although no one has proved it yet, it is thought that some numbers, like 196, never produce a palindrome. A number that never forms a palindrome through the reverse and add process is called a Lychrel number. Due to the theoretical nature of these numbers, and for the purpose of this problem, we shall assume that a number is Lychrel until proven otherwise. In addition you are given that for every number below ten-thousand, it will either (i) become a palindrome in less than fifty iterations, or, (ii) no one, with all the computing power that exists, has managed so far to map it to a palindrome. In fact, 10677 is the first number to be shown to require over fifty iterations before producing a palindrome: 4668731596684224866951378664 (53 iterations, 28-digits).

Surprisingly, there are palindromic numbers that are themselves Lychrel numbers; the first example is 4994.

How many Lychrel numbers are there below $\texttt{num}$?

In [19]:
def is_palindrome(N):
    '''
    True if N is palindrome, else False
    '''
    string = str(N)
    l = len(string)
    for i in range(len(string)//2):
        if string[i] != string[l-i-1]:
            return False
    return True

In [20]:
def lychrel_numbers(num, theor_limit):
    '''
    Number of Lychrel numbers below num (assuming
    all non-Lychrel numbers take < theor_limit
    iterations to produce a palindrome)
    '''
    tot = 0
    for N in range(num):
        for i in range(theor_limit-1):
            N = N + int(str(N)[::-1])
            if is_palindrome(N):
                break
        else:
            tot += 1
    return tot

In [21]:
lychrel_numbers(10000, 50)
# should return 249

249